In [ ]:
# IMPORT THE NECESSARY LIBRARIES
import os
import requests
import re
import pandas as pd  

In [ ]:
# OPEN GDC MANIFEST
mf_name = input('Write manifest file name: ')   #gdc_manifest.2023-09-19.txt
data = pd.read_csv(mf_name, delimiter='\t')

In [ ]:
# DIRECTORY WHERE FILES ARE SAVED
output_directory = '/Volumes/TOSHIBA EXT/4t curs/Advanced Bioinformatics/Project/Python/Files'
os.makedirs(output_directory, exist_ok=True)

In [ ]:
# LOOPING THROUGH FILES IDs, EACH FILE IS DOWNLOADED
count = 0
for _, row in data.iterrows():

    file_id = str(row[0])  
    data_endpt = 'https://api.gdc.cancer.gov/data/{}'.format(file_id)
    response = requests.get(data_endpt, headers={'Content-Type': 'application/json'})

    # REQUEST STATUS CHECK
    if response.status_code == 200:
        
        response_head_cd = response.headers.get('Content-Disposition')
        if response_head_cd:
            file_name = re.findall('filename=(.+)', response_head_cd)[0]
            output_file_path = os.path.join(output_directory, file_name)

            with open(output_file_path, 'wb') as output_file:
                output_file.write(response.content)
                count = count + 1
                print(f'Downloaded file nº {count}: {file_name}')
        else:
            print(f'Failed to get file name for file ID: {file_id}')
    else:
        print(f'Failed to download file ID: {file_id} - Status Code: {response.status_code}')